<a href="https://colab.research.google.com/github/GiorgiaAuroraAdorni/ML-bachelor-course-assignments-sp23/blob/main/assignment%201/deliverable/example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment 1
Student: Guillaume Baranzini

--- 
# IMPORTANT: all the submitted code should be in 2 cells
1) How you trained, evaluated and saved your model
2) How to load your model from a file, load the data and evaluate the model. Cell 2) should be running independently (even if cell 1 is not run)

In [11]:
import pickle
import io
import requests
import numpy as np
import pandas as pd 
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from keras.models import Sequential
from keras.layers import Dense
import matplotlib.pyplot as plt

# Load data 
url = 'https://drive.switch.ch/index.php/s/TeDwnbYsBKRuJjv/download'
response = requests.get(url)
data = np.load(io.BytesIO(response.content))

# x is a Numpy array of shape (n_samples, n_features) with the inputs
x = data.f.x
# y is a Numpy array of shape (n_samples, ) with the targets
y = data.f.y

# T1

# Split data into train and test set for x and y
x_train, x_test, y_train, y_test = train_test_split(x,y, train_size=0.90, shuffle=True, random_state=2)

print('train_data shape:', x_train.shape),print('test_data shape:', x_test.shape) # shape of the test sets + shape of the train sets

# Define the model to fit the data
def linear_function(x, theta):
    x_1 = x[:,0]
    x_2 = x[:,1]
    y = theta[0] + theta[1] * x_1 + theta[2] * x_2 + theta[3] * np.sin(x_2) + theta[4] * x_1 * x_2
    return y

# Solve the linear system with np.linalg.solve()
# Compact form
x_1 = x_train[:,0].reshape(-1,1)
x_2 = x_train[:,1].reshape(-1,1)
x_3 = np.sin(x_2)
x_4 = x_1 * x_2

ones_vector = np.ones(shape=(len(x_train),1))  # Vector of 1
X = np.hstack((ones_vector, x_1, x_2, x_3, x_4))         # The compact input

print('Input (X) shape:\t', X.shape)

theta_hat = np.linalg.solve(a=X.T.dot(X), b=X.T.dot(y_train))
# Solves ax = b with respect to x
print('Optimal parameters:\t', theta_hat)
print("f(x, θ):", theta_hat[0], "+", theta_hat[1], "*x1 +", theta_hat[2], "*x2 +", theta_hat[3], "*sin(x2) +", theta_hat[4], "*x1*x2")

# The performance test
from sklearn.metrics import mean_squared_error

train_performance = mean_squared_error(linear_function(x_train,theta_hat),y_train)
test_predictions = linear_function(x_test, theta_hat)
test_mse = mean_squared_error(test_predictions, y_test)

print("1D Train performance: \t", train_performance)
print('Test MSE:', test_mse)

# Saving the code as a pickle file
with open('linear_function.pickle', 'wb') as f:
    pickle.dump(linear_function, f)

#######################################################################################

# T2
#Define the model to fit
def non_linear_function(x, theta):
    x_1 = x[:,0]
    x_2 = x[:,1]
    y = theta[0] + theta[1] * x_1 + theta[2] * x_2 + theta[3] * np.sin(x_2) + theta[4] * x_1 * x_2
    return y

# Compact form
ones_vector = np.ones(shape=(x_train.shape[0], 1))    # Create 1900 ones
x_compact = np.hstack((ones_vector, x_1, x_2, x_3, x_4))         # The compact input

def calculate_gradient(X, Y, th):
  n = X.shape[0]
  gradient = (- 2 / n) * np.dot(X.T, (Y.reshape(-1,1) - np.dot(X, th).reshape(-1,1)))
  return gradient

def V(X, Y, theta):
  return np.mean((Y.reshape(-1,1) - np.dot(X, theta))**2)

np.random.seed(0)
theta = np.random.uniform(size=(x_compact.shape[1],1))     # initial value theta, it is random you can change it

eps = 0.0003 # step size (hyperparameter)
eps0 = 0.01
steps = 250000# number of GD steps are not precise we can use diffrent number try 

thetas_history = np.zeros(shape=(theta.shape[0], steps+1))  # save the theta values along training
loss_history = np.zeros(shape=(steps+1,1))                  # save the error values along training
thetas_history[:,0:1] = theta.copy()
loss_history[0] = V(x_compact, y_train, theta)

for i in range(1, steps+1):  
  grad = calculate_gradient(x_compact, y_train, theta)
  theta = theta - eps * grad
  mask = np.zeros((5,1))
  mask[0] = 1
  theta = theta - eps0 * grad * mask
  
  thetas_history[:,i:i+1] = theta.copy()
  loss_history[i] = V(x_compact, y_train, theta)

thetas_history = np.array(thetas_history)
loss_history = np.array(loss_history)
print('Optimal theta:', theta.T)
print(f'Mean Squared Error: {V(x_compact, y_train, theta):.8f}')


x1 = x_test[:,0].reshape(-1,1)
x2 = x_test[:,1].reshape(-1,1)
x3 = np.sin(x2)
x4 = x1 * x2

ones_vector_test = np.ones(shape=(x_test.shape[0], 1))
x_compact_test = np.hstack((ones_vector_test, x1, x2, x3, x4))
mse_test = V(x_compact_test, y_test, theta)
print(f'Mean Squared Error (test): {mse_test:.8f}')


# Saving the code as a pickle file
with open('non_linear_function.pickle', 'wb') as f:
    pickle.dump(non_linear_function, f)
    
################################################################

# T3
# Define the model
def create_model():
    model = Sequential()
    model.add(Dense(16, input_dim=2, activation='relu')) # input 
    model.add(Dense(16, activation='relu'))              # add a layer with 16 neurons
    model.add(Dense(16, activation='relu'))              # add a layer with 16 neurons
    model.add(Dense(16, activation='relu'))              # add a layer with 16 neurons
    model.add(Dense(1, activation='linear'))             # output
    model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mean_squared_error'])
    return model

# Create the model
model = create_model()

# Train the model
history = model.fit(x_train, y_train, epochs=126, batch_size=8, verbose=0, validation_split=0.1)

# Evaluate the model on test data
_, mse = model.evaluate(x_test, y_test)
print('MSE: %.4f' % mse)

# Saving the model as a pickle file
with open('model_task_3.pickle', 'wb') as f:
    pickle.dump(model, f)


train_data shape: (1800, 2)
test_data shape: (200, 2)
Input (X) shape:	 (1800, 5)
Optimal parameters:	 [ 1.24205736 -0.04503563 -0.56462773  0.47663575  0.04029328]
f(x, θ): 1.242057355919393 + -0.04503563207911075 *x1 + -0.5646277318069006 *x2 + 0.4766357540857314 *sin(x2) + 0.04029327643848045 *x1*x2
1D Train performance: 	 0.7103436190376121
Test MSE: 0.7296013997475126
Optimal theta: [[ 1.2420569  -0.04503555 -0.56462758  0.47663592  0.04029325]]
Mean Squared Error: 0.71034362
Mean Squared Error (test): 0.72960141
7/7 [==============================] - 0s 2ms/step - loss: 0.0185 - mean_squared_error: 0.0185
MSE: 0.0185


# Example on how to use baseline model:

In [12]:
# Import libraries
import joblib
import io
import requests
import numpy as np

def evaluate_predictions(y_true, y_pred):
    """
    Evaluates the mean squared error between the values in y_true and the values
    in y_pred.
    ### YOU CAN NOT EDIT THIS FUNCTION ###
    :param y_true: Numpy array, the true target values from the test set;
    :param y_pred: Numpy array, the values predicted by your model.
    :return: float, the mean squared error between the two arrays.
    """
    assert y_true.shape == y_pred.shape
    return ((y_true - y_pred) ** 2).mean()


def load_model(filename):
    """
    Loads a Scikit-learn model saved with joblib.dump.
    This is just an example, you can write your own function to load the model.
    Some examples can be found in src/utils.py.
    :param filename: string, path to the file storing the model.
    :return: the model.
    """
    model = joblib.load(filename)

    return model

# Load the data
# This will be replaced with our private test data when grading the assignment

# Load data from url
url = 'https://drive.switch.ch/index.php/s/TeDwnbYsBKRuJjv/download'
response = requests.get(url)
data = np.load(io.BytesIO(response.content))

# Alternatively yo can load the data from file
# data_path = '../data/data.npz'
# data = np.load(data_path)

# x is a Numpy array of shape (n_samples, n_features) with the inputs
x = data.f.x
# y is a Numpy array of shape (n_samples, ) with the targets
y = data.f.y

# Load the trained model
baseline_model_path = './model_task_3.pickle'
baseline_model = load_model(baseline_model_path)

# Change input
x = data.f.x

# Predict on the given samples
y_pred = baseline_model.predict(x).flatten()

############################################################################
# STOP EDITABLE SECTION: do not modify anything below this point.
############################################################################

# Evaluate the prediction using MSE
mse = evaluate_predictions(y_pred, y)
print(f'MSE on whole dataset: {mse}')

# NOTE: NOW THIS CELL IS NOT WORKING SINCE YOU NEED TO CHANGE THE INPUT.
# DO IT AND EVERYTHING RUNS SMOOTH



63/63 [==============================] - 0s 3ms/step
MSE on whole dataset: 0.01671533343495456
